In [ ]:
# Prepare data

from google.colab import drive
import os

!git clone https://github.com/alocin98/terrain-gan-public.git
%cd terrain-gan-public/code

drive.mount('/content/drive')

# Load data tools
%run datatools.py
%run models/gan.py
%run reporter.py

In [ ]:
# Load Tensorboard

%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/Terrain\ GAN/logs/detailed-inspection-128px-adapt-data

In [ ]:
# DCGAN

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def buildDCGAN(reporter, title, latent_dim, batch_size, disc_train_ratio):
    gan = GAN(reporter, title, latent_dim, disc_train_ratio)
    generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128),
        layers.Reshape((8, 8, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, kernel_size=5, padding="same", activation="sigmoid"),
    ],
        name="generator",
        )
    discriminator = keras.Sequential(
    [
        keras.Input(shape=(128, 128,1)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid"),
    ],
        name="discriminator",
        )
    gan.generator = generator
    gan.discriminator = discriminator
    return gan

In [ ]:
# Deeper DCGAN


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def buildFCCGAN(reporter, title, latent_dim, batch_size, disc_train_ratio):
    gan = GAN(reporter, title, latent_dim, disc_train_ratio)
    generator = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            layers.Dense(2 * 2 * 128),
            layers.Dense(4 * 4 * 128),
            layers.Dense(8 * 8 * 128),
            layers.Reshape((8, 8, 128)),
            layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(1, kernel_size=5, padding="same", activation="sigmoid"),
        ],
        name="generator",
        )
    discriminator = keras.Sequential(
        [
            keras.Input(shape=(128, 128,1)),
            layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(alpha=0.2),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(8 * 8 * 128),
            layers.Dense(4 * 4 * 128),
            layers.Dense(2 * 2 * 128),
            layers.Dense(1, activation="sigmoid"),
        ],
        name="discriminator",
        )
    gan.generator = generator
    gan.discriminator = discriminator
    return gan

In [ ]:
# Plain GAN


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np

def buildGAN(reporter, title, latent_dim, batch_size):
    gan = GAN(reporter, title, latent_dim)
    generator = keras.Sequential(
        [
            keras.Input(shape=(latent_dim,)),
            layers.Dense(2 * 2 * 128),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(momentum=0.8),
            layers.Dense(4 * 4 * 128),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(momentum=0.8),
            layers.Dense(8 * 8 * 128),
            layers.LeakyReLU(alpha=0.2),
            layers.BatchNormalization(momentum=0.8),
            layers.Dense(128 * 128 * 1, activation='tanh'),
            layers.Reshape((128, 128, 1))
        ],
        name="generator",
        )
    discriminator = keras.Sequential(
        [
            layers.Flatten(input_shape=(128, 128,1)),
            layers.Dense(512),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(256),
            layers.LeakyReLU(alpha=0.2),
            layers.Dense(1, activation='sigmoid')
        ],
        name="discriminator",
        )
    gan.generator = generator
    gan.discriminator = discriminator
    return gan

In [ ]:
# Train

from os import listdir
%matplotlib inline

logDir = '/content/drive/MyDrive/Terrain GAN/logs/detailed-inspection-128px/'
model_save_dir = '/content/drive/MyDrive/Terrain GAN/models/final/'


dcgan = buildDCGAN(reporter=TensorBoardReporter(logdir=logDir, print_images=100, checkpoint_filepath=model_save_dir), batch_size=4, title='DCGAN', latent_dim=128, disc_train_ratio=1)
fccgan = buildFCCGAN(reporter=TensorBoardReporter(logdir=logDir, print_images=100, checkpoint_filepath=model_save_dir),batch_size=4, title='FCCGAN', latent_dim=128, disc_train_ratio=1)
gan = buildGAN(reporter=TensorBoardReporter(logdir=logDir, print_images=100, checkpoint_filepath=model_save_dir),batch_size=4, title='GAN', latent_dim=128)

def already_tested(name):
  return False
  try:
    names = listdir(logDir)
    return any(name in string for string in names)
  except FileNotFoundError:
    return False


#fccgan-adadelta-0.001-4-meansquarederror-meansquarederror
#dcgan-adam-0.0001-4-meansquarederror-binarycrossentropy
#gan-adam-0.001-4-binarycrossentropy-binarycrossentropy

adgard = tf.keras.optimizers.Adagrad(
    learning_rate=0.001,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad"
)

adamax = tf.keras.optimizers.Adamax(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"
)

adadelta = tf.keras.optimizers.Adadelta(
    learning_rate=0.0001, rho=0.95, epsilon=1e-07, name="Adadelta"
)

sgd = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD", clipnorm=1.0
)

adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

optimizers = [(adam, 'adam-0.0001'), (adgard, 'adgard-0.0001'), (adamax, 'adadelta-0.001'), (sgd, 'sgd-0.01')]
epochs = 100

loss_fns = [(keras.losses.BinaryCrossentropy(), 'binarycrossentropy'), (keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"), 'meansquarederror')]

def getLogNameByArguments(args):
  return args.get('optimizer') +'-'+ args.get('loss_fn')

for optimizer in optimizers:
  for loss_fn in loss_fns:
      data = getData('../data/alps_hgt/', scale=2, resolution=128, step=0.1)
      args = dict(data=data[0:200],optimizer=optimizer[0],batch_size="14",epochs=epochs,loss_fn=loss_fn[0])
      names = dict(data=data[0:200],optimizer=optimizer[1],epochs=epochs,loss_fn=loss_fn[1])
      logname = 'plaingan-' + getLogNameByArguments(names) + '-scale-1-200-samples-step-1-100-epochs'
      if not already_tested(logname):
        print("training: " + logname)
        gan.train(**args, logname=logname)
        tf.keras.backend.clear_session()
      else:
        print(logname + " already tested")
      logname = 'fccgan-' + getLogNameByArguments(names) + '-scale-' + str(scale) + '-500-samples-step-' + str(step/10) + '-100-epochs-FINAL'
      if not already_tested(logname):
        print("training: " + logname)
        fccgan.train(**args, logname=logname)
        tf.keras.backend.clear_session()
      else:
        print(logname + " already tested")
        logname = 'plaingan-' + getLogNameByArguments(names) + '-scale-' + str(i)
      if not already_tested(logname):
        print("training: " + logname)
        gan.train(**args, logname=logname)
        tf.keras.backend.clear_session()
      else:
        print(logname + " already tested")
